In [1]:
import requests
import bs4
from bs4.diagnose import diagnose
import re

In [2]:
base_url = 'https://www.nbpts.org/wp-admin/admin-ajax.php'

In [3]:
page_num = 1
form_data = {'page': page_num, 'action': 'salesforce_ajax_search'}
res = requests.post(base_url,data=form_data)

In [4]:
cleaned_text = res.text.replace('\\n', '').replace('\\"', '').replace('\\/', '/').replace('<strong>', '')
soup = bs4.BeautifulSoup(cleaned_text,'lxml')

In [6]:
# soup

In [5]:
teacher_name = []
district_id = []
state = []
certificate = []
cert_start = []

teachers = soup.select('.teacher')
certifications = soup.select('.certifications')

while len(teachers) > 0 and page_num < 4: 
    
    for teacher in teachers:         
        name = teacher.h4.string
        teacher_name.append(name)

        district = teacher.h5.string
        district_id.append(district)

#         st = teacher.h6.string
#         state2.append(st)
        st = re.search('(?:(?<=, )|^)([\w ]+)$',teacher.h6.string)
        if st:
            state.append(st.group(0))
        else:
            state.append('NA')
        
    for cert in certifications:
        cert = cert.h4.string 
        certificate.append(cert)

    sdate = soup.select('p')
    cert_start.append(sdate) 
           
    page_num = page_num +1  
    form_data = {'page': page_num,'action': 'salesforce_ajax_search'}
    res = requests.post(base_url,data=form_data)    
    cleaned_text = res.text.replace('\\n', '').replace('\\"', '').replace('\\/', '/')
    soup = bs4.BeautifulSoup(cleaned_text,'lxml')
    teachers = soup.select('.teacher')
    certifications = soup.select('.certifications')

print("done")



done


In [6]:
cert_start[0]
starts = []
num = 0

while num < len(cert_start):
    for item in cert_start[num]:
        starts.append(item)
    num+=1

In [7]:
#get rid of extra info in dates
position = 0
while position < len(starts):
    starts.remove(starts[position])
    position+= 25

In [8]:
# get rid of extra info in dates part II
position = 24
while position < len(starts):
    starts.remove(starts[position])
    position+= 24

In [10]:
# break up dates in start and end dates
certificate_start = []
certificate_end = []
position = 0
while position < len(starts):
    if position%2==0:
        certificate_end.append(starts[position].text)
    else: 
        certificate_start.append(starts[position].text)
    position+=1

In [12]:
# Merge lists together as a dataframe

In [13]:
import pandas as pd

In [14]:
nb_teachers = pd.DataFrame(
    {'teacher_name': teacher_name,
     'district': district_id,
     'state': state,
     'certificate_type': certificate,
     'cert_start': certificate_start,
     'cert_end': certificate_end
    })

In [15]:
nb_teachers_wa = nb_teachers[nb_teachers['state']=="Washington"] 

In [16]:
nb_teachers_wa

,teacher_name,district,state,certificate_type,cert_start,cert_end
11,Megan Aarsvold,FEDERAL WAY SCHOOL DIST 210,Washington,Generalist/Early Childhood,Date Achieved: 12/16/2017,Valid Until: 12/16/2022
12,Haakon Aasan,NORTHSHORE SCHOOL DISTRICT 417,Washington,Science/Early Adolescence,Date Achieved: 11/15/2014,Valid Until: 11/15/2024
25,Holly Abbarno,TUMWATER SCHOOL DISTRICT 33,Washington,Literacy: Reading-Language Arts/Early and Midd...,Date Achieved: 12/16/2017,Valid Until: 12/16/2022
